In [1]:
import sys
sys.path.append('/home/sugimoto/document_summarizer')

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from omegaconf import OmegaConf
import torch
from torch.utils.data import DataLoader
from itertools import chain

In [4]:
cfg = OmegaConf.load('../config.yaml')

In [5]:
from pointer_generator_network.src.vocab import Vocab
from pointer_generator_network.src.tokenizer import Tokenizer
from pointer_generator_network.src.model import Summarizer
from pointer_generator_network.src.batch import SummBatch
from pointer_generator_network.src.batch import EncBatch
from pointer_generator_network.src.batch import BaseBatch
from pointer_generator_network.src.trainer import Trainer

In [6]:
from pointer_generator_network.src.vocab import SOS_TOKEN

/home/sugimoto/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
src_vocab = Vocab(cfg.source_vocab_path)
tgt_vocab = Vocab(cfg.target_vocab_path)
tokenizer = Tokenizer()
model = Summarizer(cfg, tgt_vocab.word2idx[SOS_TOKEN])
trainer = Trainer(model, tokenizer, src_vocab, tgt_vocab)

In [8]:
import json
json_path = "../data/test.json"
with open(json_path) as json_file:
    article_dicts = json.load(json_file)

In [39]:
idx = 189
summary = '。'.join(article_dicts[idx]['summary'])
# del article_dicts[idx]['summary']
article_dict = article_dicts[idx]
article_dict

{'id': '11378322',
 'summary': ['マナーの悪い外国人観光客が増加していると、テレビ番組が報じた',
  '桜の枝を折って注意されると、反省せずに開き直る人が多くみられる',
  '中国人はなぜか桜に触りたがり、顔の近くに引っ張って写真を撮るという'],
 'body': '関東では桜の見ごろになっているが、外国人観光客のマナーの悪さも目立つ。上野公園で桜の枝を折ったインドネシア人は「珍しいから持ち帰りたかった」、枝から花をむしりとったバングラデシュ人の子供の父親は「遠くからでなく近くで見たくて摘んだ」と言い訳する。なかでは中国人がひどい。立ち入り禁止場所に入って撮影していた中国人女性は、注意されると「入ったところには看板がなかったわ」と開き直る。枝に飛びついて引っぱり、写真を撮るのもザラで「ちょっと触れただけよ」と反省ゼロだ。\n\n\n注意されると「落ちてるの拾った」大阪城公園にドッと押し寄せる中国人はなぜか桜に触りたがり、顔の近くに引っぱって写真を撮る。「中国人はみんな自然が大好きよ。美しいものにあこがれる。中国でも触っている」（中年女性）、「春らしく躍動感を出したいの。そばに立って撮るだけなんてダサいわ」（若い女性）と意に介さない。石垣の一角が中国人のブログで、人気のスポットになっているらしい。花を摘み取り髪飾りにした女性は、平然と「落ちているのを拾ったのよ」とトボける。国民性か、言い訳というよりウソを並べる。花びらは散り、枝はボロボロだ。中国国内では木を揺らして桜吹雪を演出して写真を撮る、木を蹴ったり上ったりの画像もある。文化の違いだからといって、日本でやりたい放題にさせていいのか。ニュースデスクの笠井信輔「国策として観光客を集めるなら、見方をアピールする必要がありますよ」司会の小倉智昭「撮影できる一角を設けたら、どうなんだろう」'}

In [40]:
# example = trainer.convert_article_to_example(article_dict)
# print(tgt_vocab.decode(example.dec.input_ids))
# print(tgt_vocab.decode(example.dec.target_ids, example.enc.oov_words))

In [41]:
reference = ''.join(tokenizer(summary))
reference

'マナーの悪い外国人観光客が増加していると、テレビ番組が報じた。桜の枝を折って注意されると、反省せずに開き直る人が多くみられる。中国人はなぜか桜に触りたがり、顔の近くに引っ張って写真を撮るという'

In [42]:
# example = trainer.convert_article_to_example(article_dict)
# trainer.evaluate(article_dicts[:5])

## beam search decoding

In [43]:
example = trainer.convert_article_to_example(article_dict)
decoded_tokens = trainer.decode(example)
''.join(decoded_tokens)

'外国人観光客のマナーの悪さが目立つ、外国人観光客のマナーの悪さが目立つ。「[UNK]のところには看板がなかったわ」と「落ちたところには看板がなかった」と語った。「春らしく匹感を出したいの」とのこと'

## calculate rouge score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

In [ ]:
scorer.score(reference, ' '.join(decoded_tokens)) 